# Bootstrapping to estimate parameter ranges of nonlinear models

When we first introduced ourselves to regression, we focused the regression algorithm’s ability to find the “truth”, i.e. how close to the true parameter values does the algorithm get.

In statistics and boostrapping, the truth is (typically) unknown and there is noise associated with any given measurement. So our question becomes “how confident am I that the parameters of my model are not zero?” And as discussed in class, all kinds of challenges arise when using bootstrapping. Here, we'll try a simple example.

Let's consider one of our equations for bacterial growth:
\begin{align}
\dot{y} = \frac{y^3}{a+y^3}-by
\end{align}

where y is the concentration of bacterium and a and b are parameters that help define the rate of bacterial growth and loss, respectively. 

You have been provided with data, called "Homework11 Data.csv". Use this data for the following excercises. In this file, you'll find the time points sampled and the concentration of bacterium (y) for 5 independent experiments. We will consider data to be independent across time and experiments.

## Part 1: Plot data and perform initial fitting

In the space below, load the data and write the code to fit the parameters of our model to the data using minimize or fsolve. You may want to revisit your previous homeworks. To make things a little easier, I'll give you the first guess for your parameter. Go with (a,b) = (0.3,0.8). After fitting:
   - clearly indicate what were the parameter values and the value of the cost function you implemented.
   - Plot on one plot the data and the fitted model's response

In [ ]:
#finding the parameters for the diff eq above 

#importing the required packages 
import pandas as pd 
from scipy.optimize import minimize 
from scipy.integrate import odeint
import numpy as np
import matplotlib.pyplot as plt

bac_conc=pd.read_csv('Homework 6 Data.csv') #reading into and saving the data in the csv file 

def func(y,t,a,b): #defining the function for the diff eq where the output is the value of the function 
    fn=(y**3/(a+y**3))-(b*y);
    return fn

t_steps=bac_conc['Time'].to_numpy(); #time points from the experiments in an array 
ic=1; #initial condition set to 1 


def RMSE(params): #defining the function for root mean square error 
    a,b= params; #unknown parameters assigned to their variables 
    odefunc=odeint(func,ic,t_steps,args=(a,b)); #solves the ode with the given initial condition and input a,b
    err=bac_conc.iloc[0:,1:].to_numpy()-odefunc; #finds the difference between each time point and the calculated value
#     print(err)
    return np.sqrt(np.sum(err**2))

# print(RMSE([0.3,0.8])) #testing function 
# sol = minimize(RMSE, x0=(0.3,  0.8)); #initial guess using minimize leading to final guess below


sol = minimize(RMSE, x0=(-0.99012514,  0.08788018)); #final guess 
print(sol)
print('cost function=',14.834490,'a=',-0.99012514,'b=',0.08788018)

ode_func=odeint(func,ic,t_steps,args=(-0.99012514,  0.08788018)); #solve the diff eq with the chosen parameter values
plt.scatter(bac_conc['Time'],bac_conc['Exp1'])
plt.scatter(bac_conc['Time'],bac_conc['Exp2'])
plt.scatter(bac_conc['Time'],bac_conc['Exp3'])
plt.scatter(bac_conc['Time'],bac_conc['Exp4'])
plt.scatter(bac_conc['Time'],bac_conc['Exp5'])
plt.plot(t_steps,ode_func)

#adds labels to the plot 
plt.legend(["Exp1","Exp2","Exp3","Exp4","Exp5","Fit"])
plt.xlabel('time')
plt.ylabel('Bacteria Concentration ')
plt.title('Bacteria concentration with time for 5 experiments')



## Part 2: Use sampling with replacement to create distributions of parameters that fit the data

Our hypothesis is that both a and b are greater than 0. Use the bootstrap method to create distributions of estimates of our parameter values. We can do this by, **for each iteration:**
   - Resample the data at each time point using sample with replacement (search online how to do).
   - Use minimize to refit the model to the resampled data
   - Save the fitted parameter estimates and the value of the cost function.
   
Do this for at least 200 iterations. HINT: this code may take a long time to run. It would be best to get the code working with only a few iterations. And once happy, then try 200. After completing, plot histograms of the fitted parameter values and cost function values. Figures must be clearly labeled and obvious to read. 

Based on these data, what is the p value for the null hypothesis that a and b are equal to zero? Calculate the one sided p value, based on the idea that we expect them to be greater than zero.
   

In [ ]:
from sklearn.utils import resample #import the package to resample with replacement 

def RMSE2(params2):
    a2,b2= params2;
    odefunc2=odeint(func,ic,t_steps,args=(a2,b2));
    err2=x-odefunc2;
    return np.sqrt(np.sum(err2**2))

a_sample=[]; #initializing empty vectors to place in the paramter values and the cost function 
b_sample=[];
costfunc=[];

for k in range(0,200): #looping through 200 times 
    x=resample(bac_conc.iloc[0:,1:].to_numpy()); #resampling with replacement for each experiment at each time point 
    sol2 = minimize(RMSE2, x0=(0.3,0.8)); #using the initial guess to then obtain parameter estimates 
    a=sol2.x[0]; # estimate for a 
    b=sol2.x[1]; #estimate for b 
    RMS=RMSE2([a,b]);
    a_sample.append(a);
    b_sample.append(b);
    costfunc.append(RMS);

plt.hist(a_sample,bins=50,density=True);
plt.xlabel('a')
plt.ylabel('Probability Density')
plt.title('Distribution for parameter a')

In [ ]:
plt.hist(b_sample,bins=50,density=True);
plt.xlabel('b')
plt.ylabel('Probability Density')
plt.title('Distribution for parameter b')

In [ ]:
plt.hist(costfunc,bins=50,density=True);
plt.xlabel('RMSE')
plt.ylabel('Probability Density')
plt.title('Distribution for RMSE')

p_a=len([x for x in a_sample if x>0])/200; #finding the number of times a >0 divided by the total samples 
p_b=len([x for x in b_sample if x>0])/200; # finding the number of times b >0 divided by the total samples 
p_ab1=len([x for x in a_sample if x==0])/200; 
p_ab2=len([x for x in b_sample if x==0])/200;

print('p values for a and b greater than 0 are: ',p_a,',',p_b,'respectively','p value for a and b=0 is',p_ab1+p_ab2)

## Part 3 Analyze fit results

One a single figure, plot the model's response vs time for all 200 estimates of the parameter values. Are the fits reasonable? Should any of the fits be discounted? Why so? Does this change how you define your p values above? **Defend your answer here and include any plots as necessary:**



In [ ]:
for i in range(0, 200): #looping through the 200 parameter values to solve the diff eq at each and plot a new fit 
    ode_func2=odeint(func,ic,t_steps,args=(a_sample[i],  b_sample[i]));
    plt.plot(t_steps,ode_func2,color="blue")
    
plt.plot(t_steps, ode_func,color='red')
plt.xlabel('time')
plt.ylabel('Bacteria concentration')
plt.title('Plots of fit for bacteria concentration with original fit in red')


Some of the fits are close to the one obtained in problem 1 while a few others are much further away from an appropriate fit, these could be discounted because they do not represent the original data and are either greater than or less than the average at each time point (from observation). 

Data for the project: Looking at COVID county deaths with county age and educational attainment 